In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_california_housing

#Independent data


In [194]:
data=fetch_california_housing()
print(data.DESCR)
df=pd.DataFrame(data = data.data, columns= data.feature_names)
df.head()


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [195]:
#Dependent Data
df['Target'] = data.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [196]:
# EDA
!pip install numpy==1.26.4
!pip install sweetviz


In [198]:
import sweetviz as sv  #does EDA
report = sv.analyze(df)
report.show_html("./report.html")

                                             |          | [  0%]   00:00 -> (? left)

Report ./report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [199]:
#DATA PREPROCESSING
# Feature Engineering
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")  #api will give the location of the city for the given coordinates#


In [200]:
from geopy.geocoders import Nominatim
# Re-initialize geolocator with a more specific user agent to avoid potential blocks
geolocator = Nominatim(user_agent="colab_california_housing_analyzer")
geolocator.reverse("37.88" + " , " + "-122.23")

Location(Grizzly Peak Boulevard, Oakland, Alameda County, California, 94720, United States, (37.8803854, -122.2301805, 0.0))

In [201]:
from types import NoneType
def location(cord):
  Latitude = str(cord[0])
  Longitude = str(cord[1])

  location = geolocator.reverse(Latitude+","+Longitude).raw['address']  #raw returns a dictionary

  #if values are missing replace by a empty string

  if location.get('Road') is None:
    location['Road'] = None

  if location.get('County') is None:
    location['County'] = None

  loc_update['County'].append(location['County'])
  loc_update['Road'].append(location['Road'])


In [202]:


'''loc_update = {"County":[],
              "Road":[],
              "Neighbourhood":[]}

for i,cord in enumerate(df.iloc[:,6:-1].values):

  location(cord)

  pickle.dump(loc_update, open('loc_update.pickle','wb'))

  if i%100==0:
    print(i)'''


'loc_update = {"County":[],\n              "Road":[],\n              "Neighbourhood":[]}\n\nfor i,cord in enumerate(df.iloc[:,6:-1].values):\n\n  location(cord)\n\n  pickle.dump(loc_update, open(\'loc_update.pickle\',\'wb\'))\n\n  if i%100==0:\n    print(i)'

In [ ]:
import pickle

loc_update = pickle.load(open("/content/loc_update (1).pickle","rb"))
loc_update['county']


In [204]:
loc = pd.DataFrame(loc_update)
loc.head()

,county,road
0,Alameda County,Panoramic Way
1,Alameda County,Caldecott Lane
2,Alameda County,Grove Shafter Freeway
3,Alameda County,Florio Street
4,Alameda County,Florio Street


In [205]:
loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   county  15262 non-null  object
 1   road    19498 non-null  object
dtypes: object(2)
memory usage: 322.6+ KB


In [206]:
# add the new features to our dataframe

for i in loc_update.keys():
  df[i] = loc_update[i]

df = df.sample(axis = 0, frac = 1)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target,county,road
4852,1.2300,47.0,5.367347,1.008163,785.0,3.204082,34.03,-118.31,1.384,None,West 28th Street
9187,6.6916,2.0,9.769984,1.440457,1787.0,2.915171,34.38,-118.61,3.295,None,Pico Canyon Service Road
4373,4.7132,50.0,5.055024,0.952153,793.0,1.897129,34.10,-118.27,3.046,None,Redrock Court
15605,6.1481,11.0,7.095238,1.034799,1672.0,3.062271,32.80,-116.80,2.746,San Diego County,Sequan Truck Trail
8672,6.0308,38.0,5.857778,1.053333,547.0,2.431111,33.82,-118.38,4.167,None,Avenue E


In [207]:
# drop lat,long
df=df.drop(labels=["Latitude","Longitude"],axis=1)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Target,county,road
4852,1.2300,47.0,5.367347,1.008163,785.0,3.204082,1.384,None,West 28th Street
9187,6.6916,2.0,9.769984,1.440457,1787.0,2.915171,3.295,None,Pico Canyon Service Road
4373,4.7132,50.0,5.055024,0.952153,793.0,1.897129,3.046,None,Redrock Court
15605,6.1481,11.0,7.095238,1.034799,1672.0,3.062271,2.746,San Diego County,Sequan Truck Trail
8672,6.0308,38.0,5.857778,1.053333,547.0,2.431111,4.167,None,Avenue E


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20640 entries, 4852 to 13133
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Target      20640 non-null  float64
 7   county      15262 non-null  object 
 8   road        19498 non-null  object 
dtypes: float64(7), object(2)
memory usage: 1.6+ MB


In [209]:
#using classification algorithm to fil the missing categorical values
#applying logistic regression to find the missing value

missing_idx=[]

for i in range(df.shape[0]):

  if df['road'][i] is None:
    missing_idx.append(i)

# Independent Parameters
missing_road_X_train = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0])  if i not in missing_idx])

# Dependent Parameters
missing_road_y_train = np.array([df['road'][i] for i in range(df.shape[0]) if i not in missing_idx])

missing_road_X_test = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0]) if i in missing_idx])



In [210]:
from sklearn.linear_model import SGDClassifier

# model initialization

model_1= SGDClassifier()

# model Training

model_1.fit(missing_road_X_train,missing_road_y_train)

missing_road_y_pred = model_1.predict(missing_road_X_test)


In [211]:
# Add the model back to the data frame

for n,i in enumerate(missing_idx):
  df['road'][i] = missing_road_y_pred[n]

/tmp/ipython-input-4165069012.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['road'][i] = missing_road_y_pred[n]
/tmp/ipython-input-4165069012.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [212]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['road'] = le.fit_transform(df['road'])    #converted the categorical data of road into numerical data

In [213]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Target,county,road
4852,1.2300,47.0,5.367347,1.008163,785.0,3.204082,1.384,None,8487
9187,6.6916,2.0,9.769984,1.440457,1787.0,2.915171,3.295,None,6113
4373,4.7132,50.0,5.055024,0.952153,793.0,1.897129,3.046,None,6442
15605,6.1481,11.0,7.095238,1.034799,1672.0,3.062271,2.746,San Diego County,7025
8672,6.0308,38.0,5.857778,1.053333,547.0,2.431111,4.167,None,588


In [214]:
#doing the same thing for county


missing_idx=[]

for i in range(df.shape[0]):

  if df['county'][i] is None:
    missing_idx.append(i)

# Independent Parameters
missing_county_X_train = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0])  if i not in missing_idx])

# Dependent Parameters
missing_county_y_train = np.array([df['county'][i] for i in range(df.shape[0]) if i not in missing_idx])

missing_county_X_test = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0]) if i in missing_idx])

from sklearn.linear_model import SGDClassifier

# model initialization

model_1= SGDClassifier()

# model Training

model_1.fit(missing_county_X_train,missing_county_y_train)

missing_county_y_pred = model_1.predict(missing_county_X_test)

# Add the model back to the data frame

for n,i in enumerate(missing_idx):
  df['county'][i] = missing_county_y_pred[n]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['county'] = le.fit_transform(df['county'])    #converted the categorical data of county into numerical data

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['county'][i] = missing_county_y_pred[n]
/tmp/ipython-input-1800166681.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['county'][i] = missing_county_y_pred[n]
/tmp/ipython-input-1800166681.py:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical ex

In [215]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Target,county,road
4852,1.2300,47.0,5.367347,1.008163,785.0,3.204082,1.384,35,8487
9187,6.6916,2.0,9.769984,1.440457,1787.0,2.915171,3.295,35,6113
4373,4.7132,50.0,5.055024,0.952153,793.0,1.897129,3.046,40,6442
15605,6.1481,11.0,7.095238,1.034799,1672.0,3.062271,2.746,39,7025
8672,6.0308,38.0,5.857778,1.053333,547.0,2.431111,4.167,40,588


In [226]:
#Understand which model to use
# Regression model Random Forest

y=df.iloc[:,-3].values
df=df.drop(labels=["Target"], axis=1)
X=df.iloc[:,:].values

In [227]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,county,road
4852,1.2300,47.0,5.367347,1.008163,785.0,3.204082,35,8487
9187,6.6916,2.0,9.769984,1.440457,1787.0,2.915171,35,6113
4373,4.7132,50.0,5.055024,0.952153,793.0,1.897129,40,6442
15605,6.1481,11.0,7.095238,1.034799,1672.0,3.062271,39,7025
8672,6.0308,38.0,5.857778,1.053333,547.0,2.431111,40,588


In [228]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [229]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor()

In [230]:
# Model Prediction

y_pred = model.predict(X_test)


In [231]:
# model accuracy

from sklearn.metrics import r2_score

r2_score(y_test, y_pred)*100



74.6720179566956

In [232]:
# add our own data
inp =np.array([1.2300,	47.0,	5.367347,	1.008163,	785.0,	3.204082,		35,	8487])


In [233]:
inp= inp.reshape(1,-1)

In [237]:
out=model.predict(inp)
out

array([1.05401])

In [238]:
with open('model.pkl','wb') as file:
  pickle.dump(out, file)